# Ye-Net_Final

# IMPORTACIÓN DE LIBRERIAS

In [ ]:
from scipy import misc, ndimage, signal
import numpy
import numpy as np
import random
import ntpath
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras import optimizers 
from keras import regularizers
import tensorflow as tf
import cv2
from keras.layers import Lambda, Layer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.layers.core import Reshape
from keras import optimizers
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras import regularizers
from time import time
from keras.models import load_model
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.optimizers import SGD,RMSprop,adam,Adagrad,Nadam,Adamax,Adam,Adadelta,RMSprop
from keras.metrics import categorical_crossentropy
from keras import optimizers 
from keras import regularizers
from keras.layers import Lambda, Layer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.initializers import Constant, RandomNormal, glorot_normal
from keras.layers.core import Reshape
from keras import optimizers
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.callbacks import TensorBoard
from time import time
from random import randint
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import random
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from numpy import *
from PIL import Image
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import tensorflow as tf
import sys
import glob
from scipy import misc, ndimage, signal
import numpy
import random
import ntpath
import skimage
from skimage.util.shape import view_as_blocks, view_as_windows
import tensorflow as tf
from tensorflow import clip_by_value
import matplotlib as mpl
from keras import optimizers 
from keras import regularizers
from keras.layers.core import Reshape
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from tensorflow import clip_by_value
import time
from matplotlib import pyplot as plt
from IPython import display
from tensorflow.keras import layers, regularizers, initializers, activations

# TRABAJO SOBRE LA CPU DADO QUE LA GPU ESTÉ OCUPADA

In [ ]:
#USANDO LA CPU

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# LECTURA DE LAS IMÁGENES

# Lectura forma 1

Ésta es la forma estandar que hemos venido usando para leer las imágenes y agruparlas en bloques para que sean usadas en la red, divididas tanto en entrenamiento como en validación y test. Y las etiquetas generadas [0 1] [1 0] para ser usadas con una función de activación sotwmax con una salida de dos clases o neuronas. 

Aquí train2 significa que los 4000 pares cover-stego se le aumentó con las 10000 de la base de datos BOWS2, en éste caso la base de datos se ha venido trabajando con el algoritmo esteganográfico S-UNIWARD con una carga de o.4bpp

In [ ]:
n=256
def load_images(path_pattern):
    files=glob.glob(path_pattern)
    X=[]
    for f in files:
        I = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
        #I=np.float32(I)
        patches = view_as_blocks(I, (n, n))
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                X.append( [ patches[i,j] ] )
    X=numpy.array(X)
    return X
#S-UNIWARD 0.4bpp

#Train Images
Xc = load_images('/media/ia/Datos1/DocReinel/DATABASES/BRASbase/train2/cover/*.pgm')
Xs = load_images('/media/ia/Datos1/DocReinel/DATABASES/BRASbase/train2/stego/*.pgm')
#Validation Images
Yc = load_images('/media/ia/Datos1/DocReinel/DATABASES/BRASbase/valid/cover/*.pgm')
Ys = load_images('/media/ia/Datos1/DocReinel/DATABASES/BRASbase/valid/stego/*.pgm')
#Test Images
Zc = load_images('/media/ia/Datos1/DocReinel/DATABASES/BRASbase/test/cover/*.pgm')
Zs = load_images('/media/ia/Datos1/DocReinel/DATABASES/BRASbase/test/stego/*.pgm')

X = (numpy.vstack((Xc, Xs)))
Y = (numpy.vstack((Yc, Ys)))
Z = (numpy.vstack((Zc, Zs)))

Xt = (numpy.hstack(([0]*len(Xc), [1]*len(Xs))))
Yt = (numpy.hstack(([0]*len(Yc), [1]*len(Ys))))
Zt = (numpy.hstack(([0]*len(Zc), [1]*len(Zs))))

Xt = np_utils.to_categorical(Xt, 2)
Yt = np_utils.to_categorical(Yt, 2)
Zt = np_utils.to_categorical(Zt, 2)

idx=np.arange(len(X))
random.shuffle(idx)

X=X[idx]
Xt=Xt[idx]

X=np.rollaxis(X,1,4)  #channel axis shifted to last axis
print (X.shape)
Y=np.rollaxis(Y,1,4)  #channel axis shifted to last axis
print (Y.shape)
Z=np.rollaxis(Z,1,4)  #channel axis shifted to last axis
print (Z.shape)

# Lectura forma 2

Con el objetivo de que la lectura sea muchísimo más rápida se ha guardado la base de datos en archivos .npy como se podrá ver en la siguiente celda

In [ ]:
#No compilar
#Guardamos los datos para luego leerlos muchisimo más rápido
PATH = "/media/ia/Datos1/DocReinel/DATABASES/BRASbase/S-Uniward0_4bpp_14000train_1000valid_5000test_pair_cs/"

#Train
np.save(PATH+'X.npy', X)
np.save(PATH+'Xt.npy', Xt)
#Valid
np.save(PATH+'Y.npy', Y)
np.save(PATH+'Yt.npy', Yt)
#Test
np.save(PATH+'Z.npy', Z)
np.save(PATH+'Zt.npy', Zt)

Luego sólo hay que cargar los archivos .npy los cuales lo hacen extremadamente rápido comparado con la forma de lectura 1

In [ ]:
#Ruta de las imágenes
PATH = "/media/ia/Datos1/DocReinel/DATABASES/BRASbase/S-Uniward0_4bpp_14000train_1000valid_5000test_pair_cs/"

#Train
X = np.load(PATH+'X.npy')
Xt= np.load(PATH+'Xt.npy')
#Valid
Y = np.load(PATH+'Y.npy')
Yt= np.load(PATH+'Yt.npy')
#Test
Z = np.load(PATH+'Z.npy')
Zt= np.load(PATH+'Zt.npy')

#Tamaños de los datos y de las etiquetas
print (X.shape)
print (Xt.shape)
print (Y.shape)
print (Yt.shape)
print (Z.shape)
print (Zt.shape)

# LECTURA Y/O CARGA DE LOS 30 FILTROS SRM

In [ ]:
##################################################SRM FILTER
srm_weights = np.load('SRM_Kernels.npy')
biasSRM=numpy.ones(30)
print (srm_weights.shape)
##################################################TLU ACTIVATION FUNCTION
T3 = 3;
def TLU3(x):
    tlu3 = K.tanh(x)*T3
    return tlu3
##################################################


# CREACIÓN DE LA FUNCIÓN TRAIN

@Entradas

Modelo diseñado.

El valor del batch_size.

La cantidad de épocas.

El nombre del modelo.

@Salidas

Entrenar el modelo con los parametros ingresados.

Guardar los modelos por épocas.

Al final evaluar el modelo con la última época entrenada sobre los datos de prueba o Test.

Guardar gráficos de las precisiones obtenidas en entrenamiento y validación, así como de la perdida.

In [ ]:
def train(model, batch_size, epochs, model_name=""):
    log_dir="logs/"+model_name+"_"+"{}".format(time())
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir)
    filepath = log_dir+"/saved-model-{epoch:02d}-{val_accuracy:.2f}.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max')
    model.reset_states()

    history=model.fit(X, Xt, epochs=epochs, 
                      callbacks=[tensorboard,checkpoint], 
                      batch_size=batch_size,validation_data=(Y, Yt),verbose=1)
    
    metrics = model.evaluate(Z, Zt, verbose=0)
    
    #Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.grid()
    plt.savefig('./Accuracy_YeNet.pdf')
    plt.show()
    
    #Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.grid()
    plt.savefig('./Loss_YeNet.pdf')
    plt.show()
    
    return {k:v for k,v in zip (model.metrics_names, metrics)}

# CREACIÓN DEL MODELO

# MODELO FINAL, MEJORES RESULTADOS

In [ ]:
def get_conv_model_A(num_classes, img_size=256, compile=True):
    tf.keras.backend.clear_session()
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    
    #Layer 1
    layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layer1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    layers = tf.keras.layers.Concatenate()([layer1, layer1, layer1])
    #Layer 2
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 3
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 4
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    
    #Layer 5
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    layers = tf.keras.layers.Concatenate()([layers, layers, layers])
    layers = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
    
    #Layer 10, FC, Softmax
    layers = tf.keras.layers.Dense(128,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(64 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(32 ,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) 
    if compile:
        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

# GENERACIÓN DEL MODELO Y GUARDADO EN model

In [ ]:
model = get_conv_model_A(2)

# VISUALIZACIÓN GRÁFICA DEL MODELO

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

# ENTRENAMIENTO DEL MODELO

In [ ]:
train(model, batch_size=64, epochs=30, model_name="YeNet")

# LUEGO DE HABERLO ENTRENADO

# Cargamos el mejor modelo para evaluar sobre la mejor época en Test

Esto se hace pues en el paper de Zhu-Net los autores mencionan que cargan el mejor modelo obtenido sobre determinada época y son ésos los valores que presentan, el verbose=0, es bueno usarlo en TensorFlow 2, debido a que de lo contrario saldrán un montón de barritas de carga

In [ ]:
model = tf.keras.models.load_model('/media/ia/Datos1/DocReinel/Steganalysis/Ye-net/Ye-Net-Keras-TensorFlow/logs/YeNet_PRUEBA2_DATA_1581272457.8803444/saved-model-28-0.81.hdf5', custom_objects={'TLU3':TLU3}, compile=True)

In [ ]:
loss,accuracy = model.evaluate(Z,Zt,verbose=0)
print(f'Loss={loss:.4f} y Accuracy={accuracy:0.4f}')

# VISUALIZACIÓN DE LOS FILTROS

In [ ]:
#Filtros de las capas convolucionales
# summarize filter shapes
for layer in model.layers:
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # get filter weights
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)

In [ ]:
weights = model.get_weights()
for i in weights:
    print(i.shape)

In [ ]:
#Función para ver los filtros
def display_filters(w, figsize=(15,15)):
    plt.figure(figsize=figsize)
    w = (w-np.min(w))/(np.max(w)-np.min(w))
    cont=0
    for i in range(w.shape[-1]):
        for j in range(w.shape[-2]):
            #print(w.shape[-2],w.shape[-1],cont)
            cont=cont+1
            plt.subplot(w.shape[-2],w.shape[-1],cont)
            plt.imshow(w[:,:,0,i], interpolation="none", cmap='gray')
            plt.axis("off")


In [ ]:
#Filtro 1
w0 = model.get_weights()[0]
print (w0.shape)
type(display_filters(w0))

In [ ]:
#Filtro 2
w0 = model.get_weights()[5]
print (w0.shape)
type(display_filters(w0))

In [ ]:
#Filtro 3
w0 = model.get_weights()[10]
print (w0.shape)
type(display_filters(w0))

In [ ]:
#Filtro 4
w0 = model.get_weights()[15]
print (w0.shape)
type(display_filters(w0))

In [ ]:
#Filtro 5
w0 = model.get_weights()[20]
print (w0.shape)
type(display_filters(w0))

# ADICIONALEMTE TENEMOS PRUEBAS Y MODELOS PREVIOS A LA YE-NET COMPLETAMENTE MEJORADA

# Según un paper al usar sólo 10 SRM elegidos se lograba mejores resultados, lo cual no fué así en ésta arquitectura

VISUALIZACIÓN GRÁFICA DE UN FILTRO, Y DE LOS 30SRM

In [ ]:
srm_weights = np.load('SRM_Kernels.npy')
def display_filters(w, figsize=(100,100)):
    plt.figure(figsize=figsize)
    w = (w-np.min(w))/(np.max(w)-np.min(w))
    cont=0
    for i in range(w.shape[-1]):
        for j in range(w.shape[-2]):
            cont=cont+1
            plt.subplot(w.shape[-2],w.shape[-1],cont)
            plt.imshow(w[:,:,0,i], interpolation="none", cmap='gray')
            plt.axis("off")
            
display_filters(srm_weights[:,:,:,0:1])
print(srm_weights[:,:,:,1:2].shape)  
display_filters(srm_weights)
print(srm_weights.shape)  

EXTRACIÓN DE LOS FILTROS QUE DECIA EN EL PAPER FUERON LOS MEJORES PARA LA ARQUITECTUTA. SE MUESTRAN VISUALMENTE Y SE GUARDAN EN UN ARCHIVO .npy

In [ ]:
a=srm_weights[:,:,:,10:12]
b=srm_weights[:,:,:, 8:10]
c=srm_weights[:,:,:,14:15]
e=srm_weights[:,:,:,16:17]
f=srm_weights[:,:,:,12:14]
g=srm_weights[:,:,:,23:24]
h=srm_weights[:,:,:,25:26]
SRM10 = np.concatenate((a,b,c,e,f,g,h),axis=3)

display_filters(SRM10)
print(SRM10.shape)
np.save('10SRM_Kernels.npy', SRM10)

SE PROBARON DISTINTAS CONFIGURACIONES CON LOS 10SRM ELEGIDOS, UNA DE ELLAS ES LA MOSTRADA A CONTINUACIÓN QUE CONSISTE EN CONCATENAR LOS 10SRM PARA TENER 30SRM PERO YA SÓLO DE LOS ELEGIDOS

In [ ]:
SRM30full=np.concatenate((SRM10[:,:,:,:],SRM10[:,:,:,:],SRM10[:,:,:,:]),axis=3) 
print(SRM30full.shape)

display_filters(SRM30full)
srm_weights = SRM30full
biasSRM=numpy.ones(30)

UNA PRUEBA INTERESANTE TAMBIÉN FUÉ UNIR LOS 30SRM ORIGINALES CON LOS 10SRM ELEGIDOS, SIN EMBARGO LOS RESULTADOS NO MEJORARON

In [ ]:
#40SRM
##################################################SRM FILTER
srm_weights10 = np.load('10SRM_Kernels.npy')
print (srm_weights10.shape)
srm_weights30 = np.load('SRM_Kernels.npy')
print (srm_weights30.shape)

srm_weights40 = np.concatenate((srm_weights10,srm_weights30),axis=3)
print(srm_weights40.shape)

biasSRM40=numpy.ones(40)

# Se hizo pruebas mediante la inicialización con los filtros de Alex-Net

En una de las pruebas se usó la siguiente primera capa: 

layers = tf.keras.layers.Conv2D(96, (11,11), kernel_initializer=ALEX_FILTER, trainable=False, strides=(1,1), activation=TLU3)(inputs) #Alex-Net

Se probó extrayendo casa uno de los niveles RGB, y se probó, sin embargo no se obtuvo resultados

In [ ]:
#Cargamos el filtro de la primera convolución de Alex-Net
import mlutils
alex_weights = np.load('/media/ia/Datos1/DocReinel/Steganalysis/Ye-net/Ye-Net-Keras-TensorFlow/Otros/Alex_Filter/dict_alexnet_filters.npy', allow_pickle=True, encoding='bytes').item()
Alex=alex_weights["conv1"][0]
#Alex #See the filter values
print(Alex.shape)
mlutils.display_imgs(Alex)

In [ ]:
#Guardamos un nuevo .npy sólo con los filtros de la convolución 1 RGB
data_array = np.array(Alex, dtype=float32)
np.save('conv1_alexnet_filter_(11,11,3,96).npy', data_array)
data_array = np.load('conv1_alexnet_filter_(11,11,3,96).npy')

def ALEX_FILTER(shape, dtype=None, partition_info=None):
    filAlex=data_array
    return filAlex

In [ ]:
x=data_array[:,:,2,:] 
print(x.shape)
Alex_Fil = np.expand_dims(x, axis=2)

#Guardamos un nuevo .npy sólo con los filtros de la convolución 1 UN CANAL
Alex_Filter = np.array(Alex_Fil, dtype=float32)
np.save('conv1_alexnet_filter_(11,11,1,96).npy', Alex_Filter)
Alex_Filter = np.load('conv1_alexnet_filter_(11,11,1,96).npy')

print(Alex_Filter.shape)
def ALEX_FILTER(shape, dtype=None, partition_info=None):
    filAlex=Alex_Filter
    return filAlex

In [ ]:
def display_filters(w, figsize=(100,100)):
    plt.figure(figsize=figsize)
    w = (w-np.min(w))/(np.max(w)-np.min(w))
    cont=0
    for i in range(w.shape[-1]):
        for j in range(w.shape[-2]):
            #print(w.shape[-2],w.shape[-1],cont)
            cont=cont+1
            plt.subplot(w.shape[-2],w.shape[-1],cont)
            plt.imshow(w[:,:,0,i], interpolation="none", cmap='gray')
            plt.axis("off")
print(Alex_Filter.shape)            
display_filters(Alex_Filter)

# MODELO CUANDO SE USÓ LA TLU

In [ ]:
#74% PRECISIÓN
def get_conv_model_A(num_classes, img_size=256, compile=True):
    tf.reset_default_graph()
    tf.keras.backend.clear_session()
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    
    #Layer 1
    layers = tf.keras.layers.Conv2D(30, (5,5), kernel_initializer=SRM, strides= (1,1), trainable=False)(inputs)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    tf.clip_by_value(layers, clip_value_min=-tlu_threshold3, clip_value_max=tlu_threshold3, name='TLU_3')
    
    #Layer 2
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 3
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 4
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    
    #Layer 5
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 6
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 7
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 8
    layers = tf.keras.layers.Conv2D(16, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 9
    layers = tf.keras.layers.Conv2D(16, (3,3), strides= (3,3), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 10, FC, Softmax
    layers = tf.keras.layers.Flatten()(layers)
    layers = tf.keras.layers.Dense(256, activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.5)(layers)
    layers = tf.keras.layers.Dense(128, activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.5)(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = inputs, outputs=predictions)

    optimizer = tf.keras.optimizers.Adadelta(lr=0.4, rho=0.95)
    if compile:
        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

# MODELO YE-NET EN TENSORFLOW CON KERAS DE FORMA NATIVA

In [ ]:
#tf.keras 
#YE-NET IMPLEMENTADA EN tf.keras 
#DOS OPCIONES---> EN LAYER 1: 
###trainable=False[Converge rápido, perdida alta] 
###y sin usar trainable=False[Converge lento, perdida baja] 

def get_conv_model_A(num_classes, img_size=256):
    tf.reset_default_graph()
    tf.keras.backend.clear_session()
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    
    #Layer 1
    layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,bias], strides= (1,1), activation=TLU3)(inputs)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 2
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 3
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 4
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    
    #Layer 5
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 6
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 7
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 8
    layers = tf.keras.layers.Conv2D(16, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 9
    layers = tf.keras.layers.Conv2D(16, (3,3), strides= (3,3), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.contrib.layers.batch_norm(layers, decay=0.6, epsilon=0.001)
    
    #Layer 10, FC, Softmax
    layers = tf.keras.layers.Flatten()(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = inputs, outputs=predictions)

    optimizer = tf.keras.optimizers.Adadelta(lr=0.4, rho=0.95)
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

# MODELO YE-NET EN TENSORFLOW CON KERAS MODIFICADA

In [ ]:
#tf.keras
#YE-NET CON CAPA DE VALOR ABSOLUTO(Mejoras en el valor de precisión), Y CON LA CAPA DENSA DE CANCER(Mejoras en la perdida)
#Con un optimizador RMSprop(lr=0.001, rho=0.9) (Estabilidad en la converencia, sin sacrificar la perdida)
#Con batch_size=64, epochs=50
def get_conv_model_A(num_classes, img_size=256, compile=True):
    tf.reset_default_graph()
    tf.keras.backend.clear_session()
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    
    #Layer 1
    layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,bias], strides= (1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=0.001)(layers)
    
    #Layer 2
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(decay=0.9, epsilon=0.001)(layers)
    
    #Layer 3
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=0.001)(layers)
    
    #Layer 4
    layers = tf.keras.layers.Conv2D(30, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=0.001)(layers)
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    
    #Layer 5
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=0.001)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 6
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=0.001)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 7
    layers = tf.keras.layers.Conv2D(32, (5,5), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=0.001)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 8
    layers = tf.keras.layers.Conv2D(16, (3,3), strides= (1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=0.001)(layers)
    
    #Layer 9
    layers = tf.keras.layers.Conv2D(16, (3,3), strides= (3,3), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=0.001)(layers)
    
    #Layer 10, FC, Softmax
    layers = tf.keras.layers.Flatten()(layers)
    ################################################################
    layers = tf.keras.layers.Dense(128,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(64 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(32 ,activation="relu")(layers)
    ################################################################
    predictions = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    
    optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) #1 #Bueno, converge rápido, tocó el 78%en la época 40, y conservó sobre 77% USAR 70 O 75 EPOCHS
    
    if compile:
        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

# Nativo sólo con Keras

In [ ]:
#Con Keras
model = Sequential()
#Layer 1
model.add(Convolution2D(30, (5,5), weights=[srm_weights,bias], strides=(1,1), trainable=False, activation=TLU3, input_shape=(256,256,1)))
model.add(BatchNormalization(epsilon=0.001, scale=False))

#Layer 2
model.add(Convolution2D(30, (3,3), strides=(1,1), activation='relu', kernel_initializer='glorot_normal')) #use_bias=True, bias_initializer=Constant(0.2), 
model.add(BatchNormalization(epsilon=0.001, scale=False))

#Layer 3
model.add(Convolution2D(30, (3,3), strides=(1,1), activation='relu', kernel_initializer='glorot_normal')) #use_bias=True, bias_initializer=Constant(0.2), 
model.add(BatchNormalization(epsilon=0.001, scale=False))

#Layer 4
model.add(Convolution2D(30, (3,3), strides=(1,1), activation='relu', kernel_initializer='glorot_normal')) #use_bias=True, bias_initializer=Constant(0.2), 
model.add(BatchNormalization(epsilon=0.001, scale=False))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

#Layer 5
model.add(Convolution2D(32, (5,5), strides=(1,1), activation='relu', kernel_initializer='glorot_normal')) #use_bias=True, bias_initializer=Constant(0.2), 
model.add(BatchNormalization(epsilon=0.001, scale=False))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))

#Layer 6
model.add(Convolution2D(32, (5,5), strides=(1,1), activation='relu', kernel_initializer='glorot_normal')) #use_bias=True, bias_initializer=Constant(0.2), 
model.add(BatchNormalization(epsilon=0.001, scale=False))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))

#Layer 7
model.add(Convolution2D(32, (5,5), strides=(1,1), activation='relu', kernel_initializer='glorot_normal')) #use_bias=True, bias_initializer=Constant(0.2), 
model.add(BatchNormalization(epsilon=0.001, scale=False))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))

#Layer 8
model.add(Convolution2D(16, (3,3), strides=(1,1), activation='relu', kernel_initializer='glorot_normal')) #use_bias=True, bias_initializer=Constant(0.2), 
model.add(BatchNormalization(epsilon=0.001, scale=False))

#Layer 9
model.add(Convolution2D(16, (3,3), strides=(3,3), activation='relu', kernel_initializer='glorot_normal')) #use_bias=True, bias_initializer=Constant(0.2), 
model.add(BatchNormalization(epsilon=0.001, scale=False))

#Layer 10, FC, Softmax
model.add(Flatten())
model.add(Dense(2, activation='softmax')) #, kernel_initializer=RandomNormal(mean=0.0, stddev=0.01), use_bias=True, bias_initializer='zeros'

#Estructura del modelo
model.summary()
#COMPILADOR Y FIT
opt=optimizers.Adadelta(lr=0.4, rho=0.95)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# Modificado sólo con Keras

In [ ]:
#Con Keras [Modificaciones]
model = Sequential()
#Layer 1
model.add(Convolution2D(30, (5,5), weights=[srm_weights,bias], strides=(1,1), trainable=False, use_bias=True, bias_initializer='zeros', padding='valid', data_format="channels_last", input_shape=(256,256,1))) #probar quitando use_bias=True, bias_initializer='zeros' segun tensormaster
model.add(Activation(TLU3))
model.add(BatchNormalization())

#Layer 2
model.add(Convolution2D(30, (3,3), strides= (1,1), activation='relu', kernel_regularizer=regularizers.l2(5e-4), use_bias=True, bias_initializer=Constant(0.2), padding='valid', data_format="channels_last", kernel_initializer='glorot_normal')) #probar glorot_uniform
model.add(BatchNormalization())

#Layer 3
model.add(Convolution2D(30, (3,3), strides= (1,1), activation='relu', kernel_regularizer=regularizers.l2(5e-4), use_bias=True, bias_initializer=Constant(0.2), padding='valid', data_format="channels_last", kernel_initializer='glorot_normal'))
model.add(BatchNormalization())

#Layer 4
model.add(Convolution2D(30, (3,3), strides= (1,1), activation='relu', kernel_regularizer=regularizers.l2(5e-4), use_bias=True, bias_initializer=Constant(0.2), padding='valid', data_format="channels_last", kernel_initializer='glorot_normal'))
model.add(BatchNormalization())
#model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1))) #tensor-master

#Layer 5
model.add(Convolution2D(32, (5,5), strides= (1,1), activation='relu', kernel_regularizer=regularizers.l2(5e-4), use_bias=True, bias_initializer=Constant(0.2), padding='valid', data_format="channels_last", kernel_initializer='glorot_normal'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))

#Layer 6
model.add(Convolution2D(32, (5,5), strides= (1,1), activation='relu', kernel_regularizer=regularizers.l2(5e-4), use_bias=True, bias_initializer=Constant(0.2), padding='valid', data_format="channels_last", kernel_initializer='glorot_normal'))
model.add(BatchNormalization())
#model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(3, 3))) #tensor-master

#Layer 7
model.add(Convolution2D(32, (5,5), strides= (1,1), activation='relu', kernel_regularizer=regularizers.l2(5e-4), use_bias=True, bias_initializer=Constant(0.2), padding='valid', data_format="channels_last", kernel_initializer='glorot_normal'))
model.add(BatchNormalization())
#model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(4, 4))) #tensor-master

#Layer 8
model.add(Convolution2D(16, (3,3), strides= (1,1), activation='relu', kernel_regularizer=regularizers.l2(5e-4), use_bias=True, bias_initializer=Constant(0.2), padding='valid', data_format="channels_last", kernel_initializer='glorot_normal'))
model.add(BatchNormalization())

#Layer 9
model.add(Convolution2D(16, (3,3), strides= (3,3), activation='relu', kernel_regularizer=regularizers.l2(5e-4), use_bias=True, bias_initializer=Constant(0.2), padding='valid', data_format="channels_last", kernel_initializer='glorot_normal'))
model.add(BatchNormalization())

#Layer 10, FC, Softmax
model.add(Flatten()) #Probar con dos capas densas con 256 y 128 neuronas
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax', kernel_initializer=RandomNormal(mean=0.0, stddev=0.01), use_bias=True, bias_initializer='zeros', kernel_constraint=None)) #probar con activation='softmax' segun tensormaster, y quitando kernel_constraint=None, probar bias_initializer=Constant(0.0)

#Estructura del modelo
model.summary()
#COMPILADOR Y FIT
opt=optimizers.Adadelta(lr=0.4, rho=0.95)
model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])

# Pruebas sobre el optimizador

In [ ]:
#################################################################################################################
#optimizer = tf.keras.optimizers.Adadelta(lr=0.4, rho=0.95, decay=5e-6) #With 50 epochs --> 67,65%Yedroudj
#optimizer = tf.keras.optimizers.Adadelta(lr=0.4, rho=0.95, decay=0.01) #With 50 epochs --> 66,90%Yedroudj
#optimizer = tf.keras.optimizers.Adadelta(lr=0.4, rho=0.95)#funciona_bien_primeras pruebas 80 --> 77,85%Bra
#################################################################################################################
    
###optimizer=tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False) #3 #26 epochs supera Yedroudj-Net, perdida alta
###optimizer = tf.keras.optimizers.Adadelta(lr=0.6, rho=0.95) #2 #65 epochs --> 78,25%Bra
###optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) #1 #Bueno, converge rápido, tocó el 78%en la época 40, y conservó sobre 77% USAR 70 O 75 EPOCHS

# SI QUEREMOS QUE LA CURVA GENERADA DE TRAIN Y VALIDACIÓN EN LOSS Y EN PRECISIÓN SEAN ESTABLES

Para decidir que la mejor opción es un valor de 0.1 en el rate del spatial dropout se han hecho muchas pruebas, se ha tuneado el valor con los siguientes:

dropout_values = [0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6]

dropout_values = [0.03, 0.02, 0.01, 0.08, 0.065, 0.075, 0.085, 0.09]

dropout_values = [0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01, 0.08]

dropout_values = [0.1, 0.08] como candidatos finales

Así se podría al mejor modelo aplicarle spatial_dropout para ver su comportamiento

# Modelo sin reducir las capas

Epoca 125, 81.10% precisión, en la época 1296, 82.45% precisión sobre validación

In [ ]:
#5 experimento SPATIAL DROPOUT HA SIDO EL MEJOR pero ya con 500 épocas
def get_conv_model_A(num_classes, img_size=256, compile=True):
    tf.reset_default_graph()
    tf.keras.backend.clear_session()
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    
    #Layer 1
    layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 2
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 3
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 4
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    
    #Layer 5
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 6
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 7
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 8
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 9
    layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(16, (3,3), strides=(3,3), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 10, FC, Softmax
    #layers = tf.keras.layers.Flatten()(layers)
    layers = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers) #Yedroudj-Net en 12 epocas
    layers = tf.keras.layers.Dense(128,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(64 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(32 ,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) 
    if compile:
        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

model = get_conv_model_A(2)
model.summary()
train(model, batch_size=64, epochs=1300, model_name="5Dropout0.1_Ye-Net_initialLayer2_with1300epochs")

# Buena opción con del valor para el spatial dropout 100 epocas

Oscila en 80% en validación

In [ ]:
#Yedroudj-Net en la época 22
def get_conv_model_A(num_classes, img_size=256, compile=True):
    tf.reset_default_graph()
    tf.keras.backend.clear_session()
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    
    #Layer 1
    layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 2
    layers = tf.keras.layers.SpatialDropout2D(rate=0.07)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 3
    layers = tf.keras.layers.SpatialDropout2D(rate=0.07)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 4
    layers = tf.keras.layers.SpatialDropout2D(rate=0.07)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    
    #Layer 5
    layers = tf.keras.layers.SpatialDropout2D(rate=0.07)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 6
    layers = tf.keras.layers.SpatialDropout2D(rate=0.07)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 7
    layers = tf.keras.layers.SpatialDropout2D(rate=0.07)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 8
    layers = tf.keras.layers.SpatialDropout2D(rate=0.07)(layers)
    layers = tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 9
    layers = tf.keras.layers.SpatialDropout2D(rate=0.07)(layers)
    layers = tf.keras.layers.Conv2D(16, (3,3), strides=(3,3), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 10, FC, Softmax
    #layers = tf.keras.layers.Flatten()(layers)
    layers = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
    layers = tf.keras.layers.Dense(128,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(64 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(32 ,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) 
    if compile:
        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

#Yedroudj-Net en 12 epocas
model = get_conv_model_A(2)
model.summary()
train(model, batch_size=64, epochs=100, model_name="Ye-Net-BuenDropout0.07")

# MODELO V1 EL MEJOR CON TODAS LAS CONVOLUCIONALES DE YE-NET

In [ ]:
#PRECISIÓN CERCANA AL 80%
def get_conv_model_A(num_classes, img_size=256, compile=True):
   # tf.reset_default_graph()
    tf.keras.backend.clear_session()
    print ("using",num_classes,"classes")
    inputs = tf.keras.Input(shape=(img_size,img_size,1), name="input_1")
    
    #Layer 1
    layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
    layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 2
    #layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 3
    #layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 4
    #layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    
    #Layer 5
    #layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 6
    #layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 7
    #layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    layers = tf.keras.layers.AveragePooling2D((3,3), strides= (2,2))(layers)
    
    #Layer 8
    #layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(16, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 9
    #layers = tf.keras.layers.SpatialDropout2D(rate=0.1)(layers)
    layers = tf.keras.layers.Conv2D(16, (3,3), strides=(3,3), activation="relu", kernel_initializer='glorot_normal')(layers)
    layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
    
    #Layer 10, FC, Softmax
    layers = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
    layers = tf.keras.layers.Dense(128,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(64 ,activation="relu")(layers)
    layers = tf.keras.layers.Dropout(0.2)(layers)
    layers = tf.keras.layers.Dense(32 ,activation="relu")(layers)
    predictions = tf.keras.layers.Dense(num_classes, activation="softmax", name="output_1")(layers)
    
    #Compilador
    model = tf.keras.Model(inputs = inputs, outputs=predictions)
    optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) 
    if compile:
        model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

# OTROS MODELOS

# modelo interesante

In [ ]:
# Precisión máxima sobre validación 81,45%
inputs = tf.keras.Input(shape=(256,256,1), name="input_1")

#Layer 1
layer1 = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
layer1 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer1)
layer1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer1)
    

#Layer 1
layer2 = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
layer2 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer2)
layer2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer2)
    
print(layer1.shape)
print(layer2.shape)
layer3 = tf.keras.layers.add([layer1, layer2])
print(layer3.shape)

#Layer 2
layer = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer3) 
layer = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer)
layer = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer)
    
#Layer 3
layer = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer) 
layer = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer)
layer = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer)
    
#Layer 4
layer = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer) 
layer = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer)
layer = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer)
layer = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layer)
    
#Layer 5
layer = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer)
layer = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer)
layer = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer)




#Layer 10, FC, Softmax
layer4 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layer)
layer4 = tf.keras.layers.Dense(128,activation="relu")(layer4)
layer4 = tf.keras.layers.Dropout(0.2)(layer4)
layer4 = tf.keras.layers.Dense(64 ,activation="relu")(layer4)
layer4 = tf.keras.layers.Dropout(0.2)(layer4)
layer4 = tf.keras.layers.Dense(32 ,activation="relu")(layer4)
predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layer4)
    
#######################################    
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True) 
    
#######################################
    
#Compilador
model = tf.keras.Model(inputs = inputs, outputs=predictions)
optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr_schedule, rho=0.9) 
model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

# modelo tridireccional

In [ ]:
#sólo 68% de precisión
inputs = tf.keras.Input(shape=(256,256,1), name="input_1")
######################################################################YE-NET
layer1 = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
layer1 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer1)
layer1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer1)
layer1 = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer1) 
layer1 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer1)
layer1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer1)
layer1 = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer1) 
layer1 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer1)
layer1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer1)
layer1 = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer1) 
layer1 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer1)
layer1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer1)
layer1 = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layer1)
layer1 = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer1)
layer1 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer1)
layer1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer1)
layer1 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layer1)
layer1 = tf.keras.layers.Dense(128,activation="relu")(layer1)
layer1 = tf.keras.layers.Dropout(0.2)(layer1)
layer1 = tf.keras.layers.Dense(64 ,activation="relu")(layer1)
layer1 = tf.keras.layers.Dropout(0.2)(layer1)
layer1 = tf.keras.layers.Dense(32 ,activation="relu")(layer1)
layer1 = tf.keras.layers.Dense(2, activation="softmax", name="output_1Ye-Net")(layer1)

###############################################################XU-NET
layer2 = tf.keras.layers.Conv2D(30,(5,5), weights=[srm_weights,biasSRM], trainable=False, padding="same", data_format="channels_last",activation='tanh')(inputs)
layer2 = tf.keras.layers.Conv2D(16,(5,5), activation="tanh", padding="same", data_format="channels_last",kernel_initializer='glorot_normal')(layer2)
layer2 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer2)
layer2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer2)
layer2 = tf.keras.layers.AveragePooling2D((5,5),strides=2, padding="same", data_format="channels_last")(layer2)
layer2 = tf.keras.layers.Conv2D(32,(5,5), activation="tanh",padding="same", data_format="channels_last",kernel_initializer='glorot_normal')(layer2)
layer2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer2)
layer2 = tf.keras.layers.AveragePooling2D((5,5),strides=2, padding="same", data_format="channels_last")(layer2) 
layer2 = tf.keras.layers.Conv2D(64,(1,1), activation="relu", padding="same", data_format="channels_last",kernel_initializer='glorot_normal')(layer2)
layer2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer2)
layer2 = tf.keras.layers.AveragePooling2D((5,5),strides=2, padding="same", data_format="channels_last")(layer2)
layer2 = tf.keras.layers.Conv2D(128,(1,1), activation="relu",padding="same", data_format="channels_last",kernel_initializer='glorot_normal')(layer2)
layer2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer2)
layer2 = tf.keras.layers.AveragePooling2D((5,5),strides=2, padding="same", data_format="channels_last")(layer2)
layer2 = tf.keras.layers.Conv2D(256,(1,1), activation="relu",padding="same", data_format="channels_last",kernel_initializer='glorot_normal')(layer2)
layer2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer2)
layer2 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_first")(layer2)
layer2 = tf.keras.layers.Dense(128,activation="softmax",name="output_1")(layer2)
layer2 = tf.keras.layers.Dense(64,activation="softmax",name="output_2")(layer2)
layer2 = tf.keras.layers.Dense(32,activation="softmax",name="output_4")(layer2)
layer2 = tf.keras.layers.Dense(2,activation="softmax",name="output_5Xu-net")(layer2)

###############################################################YEDROUDJ-NET
layer3 = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
layer3 = tf.keras.layers.Lambda(tf.keras.backend.abs)(layer3)
layer3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer3)
layer3 = tf.keras.layers.Conv2D(30, (5,5), strides=(1,1), activation=TLU3, kernel_initializer='glorot_normal')(layer3) 
layer3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer3)
layer3 = tf.keras.layers.Conv2D(30, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer3) 
layer3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer3)
layer3 = tf.keras.layers.AveragePooling2D((5,5), strides= (2,2))(layer3)
layer3 = tf.keras.layers.Conv2D(64, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer3)
layer3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer3)
layer3 = tf.keras.layers.AveragePooling2D((5,5), strides= (2,2))(layer3)
layer3 = tf.keras.layers.Conv2D(128, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer3)
layer3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer3)
layer3 = tf.keras.layers.AveragePooling2D((5,5), strides= (2,2))(layer3)
layer3 = tf.keras.layers.Conv2D(256, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layer3)
layer3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layer3)
layer3 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layer3)
layer3 = tf.keras.layers.Dense(256, activation="relu")(layer3)
layer3 = tf.keras.layers.Dense(1024, activation="relu")(layer3)
layer3 = tf.keras.layers.Dense(2, activation="softmax", name="output_1Yedroudj-Net")(layer3)


print(layer1.shape)
print(layer2.shape)
print(layer3.shape)
predictions = tf.keras.layers.add([layer1, layer2, layer3])*(1/3)
print(predictions.shape)

#Compilador
model = tf.keras.Model(inputs = inputs, outputs=predictions)
optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) 
model.compile(optimizer=optimizer,
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

# modelo con shortcuts

In [ ]:
#No genera resultados superiores al 80% pese a su complejidad
inputs = tf.keras.Input(shape=(256,256,1), name="input_1")
    
#Layer 1
layers = tf.keras.layers.Conv2D(30, (5,5), weights=[srm_weights,biasSRM], strides=(1,1), trainable=False, activation=TLU3, use_bias=True)(inputs)
layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
layers1 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)

#Layer 2
layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
layers2 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
#layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)

#Layer 3
layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
layers3 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
#layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)

#Layer 4
layers = tf.keras.layers.Conv2D(30, (3,3), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers) 
layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
layers4 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)
#layers = tf.keras.layers.AveragePooling2D((2,2), strides= (2,2))(layers)
    
#Layer 5
layers = tf.keras.layers.Conv2D(32, (5,5), strides=(1,1), activation="relu", kernel_initializer='glorot_normal')(layers)
layers = tf.keras.layers.Lambda(tf.keras.backend.abs)(layers)
layers = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(layers)
layers5 = tf.keras.layers.GlobalAveragePooling2D(data_format="channels_last")(layers)

layers = tf.keras.layers.concatenate([layers1,layers2,layers3,layers4,layers5])
#Layer 10, FC, Softmax
layers = tf.keras.layers.Dense(128,activation="relu")(layers)
layers = tf.keras.layers.Dropout(0.2)(layers)
layers = tf.keras.layers.Dense(64 ,activation="relu")(layers)
layers = tf.keras.layers.Dropout(0.2)(layers)
layers = tf.keras.layers.Dense(32 ,activation="relu")(layers)
predictions = tf.keras.layers.Dense(2, activation="softmax", name="output_1")(layers)
        
#Compilador
model = tf.keras.Model(inputs = inputs, outputs=predictions)
optimizer=tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9) 
model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

tf.keras.utils.plot_model(model, show_layer_names=True, show_shapes=True)

<img src=https://wallpapercave.com/wp/wp3169267.jpg>